In [1]:
# For Google Colab use
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd '/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder'    
except ModuleNotFoundError:
    pass

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MLP-DeepfakeDetection-VariationalAutoencoder


In [2]:
# Imports
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from numpy.random import seed

import tensorflow as tf

import keras
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import *
from keras.applications import *
from keras import metrics
from keras.losses import binary_crossentropy
from keras import backend as K

# !pip install -U keras-tuner
# from kerastuner.tuners import RandomSearch, Hyperband
# from kerastuner.engine.hypermodel import HyperModel
# from kerastuner.engine.hyperparameters import HyperParameters
# from kerastuner import Objective

In [3]:
# Check GPU available
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Found GPU at: /device:GPU:0
Num GPUs Available:  1


In [185]:
# General model settings
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 28, 28, 3
EPOCHS = 300
DATA_GENERATOR_SEED = 1337
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1
LEARNING_RATE = 1e-3
LATENT_DIM = 1600

tf.random.set_seed(DATA_GENERATOR_SEED)
seed(DATA_GENERATOR_SEED)

# Pick dataset; DF_TYPE={'rnd-30', 'avg-30'}
DF_TYPE = 'avg-30'
TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}'
TEST_DIR = f'{TRAIN_VAL_DIR}-test'

# TRAIN_VAL_DIR += '/Celeb-real'
# TEST_DIR += '/Celeb-real'

TRAIN_VAL_DIR, TEST_DIR

('./Celeb-DF-v2/Celeb-avg-30', './Celeb-DF-v2/Celeb-avg-30-test')

In [186]:
TRAIN_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, horizontal_flip = True, fill_mode='nearest', validation_split = VALIDATION_SPLIT)
TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                    batch_size = BATCH_SIZE,
                                                    class_mode = 'binary', 
                                                    target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                    subset = 'training',
                                                    seed = DATA_GENERATOR_SEED)

VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0, validation_split = VALIDATION_SPLIT)
VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                         batch_size = BATCH_SIZE,
                                                         class_mode = 'binary', 
                                                         target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                         subset = 'validation',
                                                         seed = DATA_GENERATOR_SEED)

TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.0)
TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
                                                  batch_size = BATCH_SIZE,
                                                  class_mode = 'binary', 
                                                  target_size = (IMG_HEIGHT, IMG_WIDTH),                                
                                                  seed = DATA_GENERATOR_SEED)



Found 49942 images belonging to 2 classes.
Found 5548 images belonging to 2 classes.
Found 19638 images belonging to 2 classes.


In [140]:
# Random sampling function for latent vector, z; "variational part"
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [141]:
# # Encoder

# encoder_inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
# # 100x100x3

# x = layers.Conv2D(16, 3, strides=1, padding="same")(encoder_inputs)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 100x100x16

# x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 50x50x32

# x = layers.Conv2D(64, 3, strides=1, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 50x50x64

# x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 25x25x32

# x = layers.Flatten()(x)
# # 20000

# # Sampling
# z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
# z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)
# z = Sampling()([z_mean, z_log_var])
# # 1600

# # Defining full encoder as keras model
# encoder = keras.Model(encoder_inputs, 
#                       [z_mean, z_log_var, z], 
#                       name="encoder")

In [142]:
# # Decoder
# latent_inputs = keras.Input(shape=(LATENT_DIM,))

# x = layers.Dense(LATENT_DIM)(latent_inputs)
# x = layers.Reshape((5, 5, 64))(x)
# # 1600

# x = layers.Conv2DTranspose(32, 3, strides=5, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 25x25x32

# x = layers.Conv2DTranspose(64, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 50x50x64

# x = layers.Conv2DTranspose(32, 3, strides=1, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 50x50x32

# x = layers.Conv2DTranspose(16, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 100x100x16

# x = layers.Conv2DTranspose(3, 3, strides=1, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('sigmoid')(x)
# decoder_outputs = x
# # 100x100x3

# # Defining full decoder as keras model
# decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

In [ ]:
encoder = Sequential([
    Convolution2D(64, (3, 3), padding = 'same', input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)),
    BatchNormalization(axis = 1),
    Activation('relu'),
    Convolution2D(64, (3, 3), padding = 'same'),
    BatchNormalization(axis = 1),
    Activation('relu'),
    AveragePooling2D(),
    Convolution2D(128, (3, 3), padding = 'same'),
    BatchNormalization(axis = 1),
    Activation('relu'),
    Convolution2D(128, (3, 3), padding = 'same'),
    BatchNormalization(axis = 1),
    Activation('relu'),
    AveragePooling2D(),
    Flatten(),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dense(encoding_size),
    Activation('tanh')
])

sampling = Sequential([Dense(49, input_shape = (encoding_size,)),
                       MySampling(),
                       BatchNormalization(),
                       Activation('relu')
])



In [154]:
# Encoder

encoder_inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
# 128x128x3

x = layers.Conv2D(16, 3, strides=1, padding="same")(encoder_inputs)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 128x128x16

x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 64x64x32

x = layers.Conv2D(64, 3, strides=1, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 64x64x64

x = layers.Conv2D(64, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 32x32x64

x = layers.Conv2D(64, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
# 16x16x64

x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
x = layers.BatchNormalization(axis = 1)(x)
x = layers.Activation('relu')(x)
x

# x = layers.Flatten()(x)
# # 20000

# # Sampling
# z_mean = layers.Dense(LATENT_DIM, name="z_mean")(x)
# z_log_var = layers.Dense(LATENT_DIM, name="z_log_var")(x)
# z = Sampling()([z_mean, z_log_var])
# # 1600

# # Defining full encoder as keras model
# encoder = keras.Model(encoder_inputs, 
#                       [z_mean, z_log_var, z], 
#                       name="encoder")

<KerasTensor: shape=(None, 8, 8, 128) dtype=float32 (created by layer 'activation_238')>

In [ ]:
# Decoder
decoder = Sequential([
    Dense(49, input_shape = (encoding_size,)),
    BatchNormalization(),
    Activation('relu'),
    Reshape((7, 7, 1)),
    Convolution2D(128, (3, 3), padding = 'same'),
    BatchNormalization(axis = 1),
    Activation('relu'),
    Convolution2D(128, (3, 3), padding = 'same'),
    BatchNormalization(axis = 1),
    Activation('relu'),
    UpSampling2D(),
    Convolution2D(64, (3, 3), padding = 'same'),
    BatchNormalization(axis = 1),
    Activation('relu'),
    Convolution2D(64, (3, 3), padding = 'same'),
    BatchNormalization(axis = 1),
    Activation('relu'),
    UpSampling2D(),
    Convolution2D(3, (3, 3), padding = 'same', activation = 'tanh')
])
# latent_inputs = keras.Input(shape=(LATENT_DIM,))

# x = layers.Dense(LATENT_DIM)(latent_inputs)
# x = layers.Reshape((5, 5, 64))(x)
# # 1600

# x = layers.Conv2DTranspose(32, 3, strides=5, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 25x25x32

# x = layers.Conv2DTranspose(64, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 50x50x64

# x = layers.Conv2DTranspose(32, 3, strides=1, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 50x50x32

# x = layers.Conv2DTranspose(16, 3, strides=2, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('relu')(x)
# # 100x100x16

# x = layers.Conv2DTranspose(3, 3, strides=1, padding="same")(x)
# x = layers.BatchNormalization(axis = 1)(x)
# x = layers.Activation('sigmoid')(x)
# decoder_outputs = x
# # 100x100x3

# # Defining full decoder as keras model
# decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

In [129]:
# VAE summary
encoder.summary()
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_63 (InputLayer)           [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv2d_126 (Conv2D)             (None, 100, 100, 16) 448         input_63[0][0]                   
__________________________________________________________________________________________________
batch_normalization_158 (BatchN (None, 100, 100, 16) 400         conv2d_126[0][0]                 
__________________________________________________________________________________________________
activation_156 (Activation)     (None, 100, 100, 16) 0           batch_normalization_158[0][0]    
____________________________________________________________________________________________

In [130]:
# Build VAE model as custom (keras model class) class
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(binary_crossentropy(data, reconstruction),
                              axis=(1, 2)))
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {"loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result()}

In [132]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

(x_train, _) = TRAIN_GENERATOR.next()
(x_test, _) = VALIDATION_GENERATOR.next()
training_data = np.concatenate([x_train, x_test], axis=0)
training_data = np.expand_dims(training_data, -1).astype("float32") / 255
training_data = training_data[:, :, :, :, 0]

vae.fit(training_data,
        epochs=100,
        batch_size=BATCH_SIZE)

Epoch 1/100
2/2 [==============================] - 3s 368ms/step - loss: 117.3972 - reconstruction_loss: 115.1968 - kl_loss: 2.7996
Epoch 2/100
2/2 [==============================] - 1s 347ms/step - loss: 120.1180 - reconstruction_loss: 115.3078 - kl_loss: 3.6223
Epoch 3/100
2/2 [==============================] - 1s 342ms/step - loss: 118.3375 - reconstruction_loss: 114.1579 - kl_loss: 3.6935
Epoch 4/100
2/2 [==============================] - 1s 344ms/step - loss: 117.4119 - reconstruction_loss: 114.0985 - kl_loss: 3.0357
Epoch 5/100
2/2 [==============================] - 1s 337ms/step - loss: 117.8244 - reconstruction_loss: 114.4015 - kl_loss: 2.5090
Epoch 6/100
2/2 [==============================] - 1s 343ms/step - loss: 115.8291 - reconstruction_loss: 113.5046 - kl_loss: 2.5308
Epoch 7/100
2/2 [==============================] - 1s 341ms/step - loss: 116.1485 - reconstruction_loss: 113.4798 - kl_loss: 2.8234
Epoch 8/100
2/2 [==============================] - 1s 335ms/step - loss: 115

In [244]:
# X = np.expand_dims(x_test[0], axis=0)

# plt.figure()
# plt.imshow(X.squeeze())
# Z = vae.encoder.predict(X)
# for z in Z:
#     X_primed = vae.decoder.predict(z)
#     face = X_primed.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
#     face = np.array(face*255*225, dtype=np.uint8)
    
#     plt.figure()
#     plt.imshow(face)

%matplotlib inline

for x in x_test[:10]:
    X = np.expand_dims(x, axis=0)
    Z = vae.encoder.predict(X)[0]

    X_prime = vae.decoder.predict(Z)
    face = X_prime.reshape(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    face = np.array(face*255, dtype=np.uint8)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(5, 3))
    axes[0].imshow(X.squeeze())
    axes[1].imshow(face)
    fig.tight_layout()

ValueError: ignored

In [162]:
import numpy             as np
import matplotlib.pyplot as plt
import keras.backend     as K

from keras.models       import Sequential
from keras.layers       import Dense
from keras.layers       import BatchNormalization
from keras.layers       import Reshape
from keras.layers       import UpSampling2D
from keras.layers       import Convolution2D
from keras.layers       import Activation
from keras.layers       import Flatten
from keras.layers       import AveragePooling2D
from keras.optimizers   import Adam
from keras.datasets     import mnist